# Temperature Dependence

OpenMC is a bit different than other Monte Carlo codes in how it treats temperature depedence in a few ways. First, as we've seen already, one never has to specify "cross sections suffixes" that appear in codes that use ACE data like MCNP and Serpent. As we'll show here, temperatures can be directly specified on cells or materials. Additionally, OpenMC has the option of using special windowed multipole data that allows it to do on-the-fly Doppler broadening.

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import openmc

import sys
sys.path.append('..')
from inputs import *

For this session, we'll work with a simple pin-cell model.

In [ ]:
pincell = PinCellInputSet()

First, let's see what materials have been assigned to cells:

In [ ]:
cells = pincell.geometry.get_all_cells()
for cid, cell in cells.items():
    print(cid, cell.fill.name)

Now we can assign variables to these cells.

In [ ]:
fuel = cells[10000]
clad = cells[10001]
water = cells[10002]

Thus far, no temperatures have been assigned anywhere in the model. There are three ways we can specify temperatures:

- As a global default in `Settings.temperature['default']`.
- As a default for a material in `Material.temperature`.
- Applied to a cell as `Cell.temperature`.

Material temperatures override the global default and cell temperatures override the material defaults. In the absence of any specification, OpenMC assumes that you want room temperature (293.6 K) cross sections.

## `Settings.temperature`

The `Settings.temperature` attribute is a dictionary that takes the following keys:

- `'default'`: global default temperature in Kelvin
- `'method'`: method for handling temperature, either `'nearest'` or `'interpolation'`
- `'tolerance'`: for nearest temperature, how close a temperature can be
- `'multipole'`: Boolean value indicating whether to use multipole data

To use multipole data, we either need to set `Settings.multipoly_library` or the `OPENMC_MULTIPOLE_LIBRARY` environment variable.

## Fuel temperature coefficient

In this example, we'll calculate the fuel temperature coefficient in our pin cell model by iteratively changing the temperature. The fuel has U234, U235, U238, and O16 in it. Our multipoly library does not have data for O16, so we'll set a very high tolerance for temperature searches (effectively assuming the O16 cross section doesn't change).